In [14]:
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns

base_path = Path(r"Datasets\Smart meters in London")

### Acorn Lookup

In [12]:
path_acorn_csv = base_path / 'data' / 'informations_households.csv'
assert path_acorn_csv.exists(), f"File not found: {path_acorn_csv}"

lookup_dict = {}
lookup_df = pd.read_csv(path_acorn_csv)

# Drop all rows with Acorn not in [ACORN-A, ... , ACORN-V]
lookup_df = lookup_df[lookup_df['Acorn'].str.match(r'ACORN-[A-Z]')]
lookup_df = lookup_df[['LCLid', 'Acorn', 'Acorn_grouped']]

In [ ]:
# What we want:
# A list of shape LCLid, date, 00:00, 00:30, 01:00, 01:30, ..., 23:30
# Where the values are the energy consumption for that half hour

# What we have:
# A list of shape LCLid, timestamp, energy consumption
# Where the timestamp is in the format YYYY-MM-DD HH:MM:SS.0000000

data = {}

# LCLid,tstp,energy(kWh/hh)
# MAC000026,2011-12-07 11:00:00.0000000, 0.611 
for i in range(0,20):
    # Try to read all to see if there are any errors
    print(f'block_{i}.csv')
    df = pd.read_csv(
        f'data/block_{i}.csv', 
        dtype={'LCLid': 'str', 'tstp': 'str', 'energy(kWh/hh)': 'float'}, 
        on_bad_lines='warn',
        na_values=['Null']
    )
    df['LCLid'] = df['LCLid'].astype('str')
    df['tstp'] = pd.to_datetime(df['tstp'], format='%Y-%m-%d %H:%M:%S.%f')
    
    df.dropna(inplace=True)
    
    df['date'] = df['tstp'].dt.date
    df['time'] = df['tstp'].dt.time
    df.drop(columns=['tstp'], inplace=True)
    
    df['energy'] = df['energy(kWh/hh)'].astype('float')
    df.drop(columns=['energy(kWh/hh)'], inplace=True)
    
    df['energy'] = df['energy'] * 1000 # Convert to Wh
    df['energy'] = df['energy'].astype('int')
    
    for id, date, time, energy in df.values:
        if id not in data:
            data[id] = {}
        if date not in data[id]:
            data[id][date] = {}
        data[id][date][time] = energy

rows = []
for id, dates in data.items():
    for date, times in dates.items():
        readings = [list() for _ in range(24)]
        for time, energy in times.items():
            hour = time.hour
            readings[hour].append(energy)
        if any(len(r) == 0 for r in readings):
            print(f'Error in {id} {date}', readings)
            continue
        for i in range(24):
            if len(readings[i]) == 1:
                readings[i] = readings[i] + readings[i]
        row = [id, date] + [sum(r) for r in readings]
        rows.append(row)
print(f'Writing {len(rows)} rows')
headers=['LCLid', 'date'] + [f'{i:02}:00' for i in range(0,24)]
out_df = pd.DataFrame(rows, columns=headers)

# Keep at most 5 entries per LCLid
# out_df = out_df.groupby('LCLid').sample(5)

out_df.shape
out_df.to_csv('data/processed.csv', index=False)

In [ ]:
# Look at how many entries we have for each date
step = out_df.groupby('date').size()
# Filter out dates with less than 100 entries
step = step[step > 100]
# Pick a random date from that list
date = step.sample(1).index[0]
# Filter out all entries that are not from that date
out_df[out_df['date'] == date]

In [ ]:
day_df = pd.read_csv('data/processed.csv')
value_columns = [f'{i:02}:00' for i in range(0,24)]
# Add column for max min sum and mean
day_df['max'] = day_df[value_columns].max(axis=1)
day_df['min'] = day_df[value_columns].min(axis=1)
day_df['sum'] = day_df[value_columns].sum(axis=1)
day_df['mean'] = day_df[value_columns].mean(axis=1)
# Group by LCLid merge the columns
day_df = day_df.groupby('LCLid').agg({
    'max': 'max',
    'min': 'min',
    'sum': 'mean',
    'mean': 'mean'
})

# Merge with lookup_df on LCLid
day_df = day_df.merge(lookup_df, on='LCLid')
day_df.head()

In [ ]:
sns.pairplot(day_df, hue='Acorn')